# Social Computing - Summer 2023
# Exercise 3 - Finding Groups: Clustering Methods

Clustering aims at grouping data-points according to certain characteristics of interest. The groupings at maximizing intra-cluster cohesion and inter-cluster de-cohesion. In this exercise, we will take a look at Louvain clustering and a comparison between K-means and Gaussian Mixture Models.

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import copy
import random

## Task 3.1: Louvain Clustering 
In this section of the exercise, you will look at a clustering method which is an alternative to the Newmann Girvan Method from the lecture, the Louvain algorithm. It is an efficient algorithm for computing graph clusterings. Your task will be to implement it.

### Louvain Algorithm
The central goal of this algorithm is to sequentially merge nodes into clusters based on modularity as a local optimality criterion. Even though the algorithm is "greedy" and not guaranteed to find the best clustering, it gives quite good results for detecting communities and groups. 

We start with singletons (each node in its own cluster) and pick a node $i$. For each friend $j$ of $i$, we calculate Modularity values that result from adding $i$ to the singleton cluster $j$ (forming a new 2-cluster) or that result from adding $i$ to the non-singleton cluster that contains $j$. We merge $i$ with that $j$ for which the gain in Modularity is highest. We do this sequentially for all nodes $i$ in the graph. 

Then, each new cluster is replaced by a new node representing the cluster. Edges within the cluster are translated into self-edges of that new cluster node. We can then go back to the first step and iterate. 

Modularity for a graph with adjacency matrix $A_{ij}$ is by the difference between the number (or added weight) of intra-cluster edges in the actual graph $G(V,E)$ and  the number (or added weight) of intra-cluster edges in a random version of the graph where we keep the degrees (or added edge weights) of each node as in the original graph: 

$$Q = \frac{1}{2m}\sum_{i,j\in V}[A_{ij}-\frac{k_ik_j}{2m}]\delta(c_i, c_j)$$

where $m=1/2 \sum_{i,j\in V}A_{ij}$ and $c_n$ is the id of the cluster that node $n$ is in and $\delta(x,y)=1$ if $x=y$ and $\delta(x,y)=0$ else. 

We may as well use the following simplified [formula](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.quality.modularity.html) [1]:<br>
$$ Q = \sum_{c=1}^{n} [\frac{L_c}{m} - (\frac{k_c}{2m})^2] $$
with $L_c$ being the number of intra-community links for community c, and $k_c$ being the sum of degrees of the nodes in community c.

The algorithm's implementation thus proceeds as follows:
* (1) For each node $i$: For each neighbour $j$: Calculate modularity values for possible merges of $i$ with the cluster containing $j$. Add $i$ to the cluster with highest resulting modularity gain (if no gain is possible, $i$ stays as it is)
* (2) Merge each cluster into a new node representing the cluster. 
* Repeat (1) + (2) until no improvement can be made

**Write a Python program that computes the graph clustering with the Louvain method by completing the tasks below.** The program's input is a NetworkX Graph object. The output should be the value of the optimal modularity value found and the corresponding Graph object representing the best clustering.

### a) Modularity
In order to compare different clusterings later, the first thing to do is **implementing a function for the modularity calculation** by means of the formula given above. 

In addition to that we want to calculate the modularity gain when analyzing possible merges for a node $i$ into a cluster $c$. We get the following expression:

$$ \Delta Q_{c,i} = \frac{k_{i,c}}{m} - \frac{2\Sigma_{tot} \; k_i}{(2m)^2} $$

where $k_{i,c}$ denotes the sum of weights of edges from node $i$ to nodes in cluster $c$ and $\Sigma_{tot}$ the sum of weights of edges incident to nodes in $c$.

**Notes:**
* It may be helpful to look at the lecture's slides on the topic again for understanding modularity and clustering algorithms.
* This [website](https://towardsdatascience.com/louvain-algorithm-93fde589f58c) [2] may help you to understand more how the algorithm operates
* You are free to use NetworkX function like Graph.degree and Graph.get_edge_data but not any modularity functions etc.
* Your input graphs will always have a "weight" attribute with an assigned value.

In [ ]:
# TODO: Calculates the degree incident in node i
def calc_ki(g, i):

# TODO: Calculates the sum of the weights of all edges in the graph
def calc_m(g):

# TODO: Calculates the sum of the weights of all edges to nodes in com
def calc_tot(g, com):

# TODO: Calculates the sum of the weights of edges from node i to nodes in com
# NOT SURE: it calculates A_{ij} from the first formula
def calc_ki_in(g, i, com):

# TODO: Calculates the delta Modularity with the formula given above
# NOT SURE: It should be the [] part of the first formula
def calc_deltaMm(g, i, com, m):

# TODO: Calculates the sum of the weights of all intra-community edges in com
# It calculates L_c in the simplified formula above
def calc_lc(g, com):

# TODO: Calculates the sum of degrees of the nodes in com
# It calculates k_c in the simplified formula above
def calc_kc(g, com):

# TODO: Calculates the modularity with the formula given above
def calc_mod(g, M_com, m):


### b) Community Aggregation
After finishing the first step, all nodes belonging to the same community are merged into a single node. This step also generates self-loops which are the sum of all links inside a given community, before being collapsed into one node.

**Notes:**
* You are free to implement everything on your own (as long as the result is correct), but a possible way to solve it is by working with a dictionary containing the new partition.
* It is useful to add an attribute 'name' to the original graph's nodes in order to preserve the indices as they are reset when creating subgraphs.
* When testing your function, you can use the function `plotGraph` defined below to get a visual verification.
* `M_com` is a dictionary that contains the new community partition (e.g. M_com equals {'A': {'A', 'B'}, 'B': set(), 'C': set(), 'D': {'C', 'D'}, 'E': {'E', 'F'}, 'F': set()})

In [ ]:
def plotGraph(g):
    # Create positions of all nodes and save them
    pos = nx.spring_layout(g)

    # Draw the graph according to node positions
    nx.draw(g_new, pos, with_labels=True)

    # Create edge labels
    labels = {e: str(g.get_edge_data(*e)) for e in g.edges}

    # Draw edge labels according to node positions
    nx.draw_networkx_edge_labels(g, pos, edge_labels=labels)

    plt.show()

In [ ]:
def mergeCommunities(g, M_com):
    # TODO:


### c) Louvain Loop
The next step is to implement the core functionality. Determine the best partition based on modularity in the current graph and use your function `mergeCommunities` to merge accordingly. This is repeated until no improvement can be made. The final output is the graph with the best clustering and the corresponding modularity value.

**Notes:**
* It may be helpful to look at the lecture's slides on the topic again for understanding modularity and clustering algorithms.
* Our test graph is taken from this [website](https://www.statworx.com/de/blog/community-detection-with-louvain-and-infomap/) [4] which once again explains the algorithm [3] and you can lookup the correct partition of said graph there.
* For testing purposes you might want to use more than one graph. Remember to assign weights to your graphs edges as we did below on the Krackhardt Kite Graph.

In [ ]:
# Krackhardt Kite Graph with assigned weight=1
def krackhardt_graph():
    g = nx.krackhardt_kite_graph()
    for e in g.edges():
        g[e[0]][e[1]]["weight"] = 1
    return g

# Test graph from linked website
def test_graph():
    g = nx.Graph()
    g.add_node("A")
    g.add_node("B")
    g.add_node("C")
    g.add_node("D")
    g.add_node("E")
    g.add_node("F")
    g.add_edge("A", "B", weight=5)
    g.add_edge("A", "C", weight=4)
    g.add_edge("A", "E", weight=1)
    g.add_edge("B", "C", weight=2)
    g.add_edge("C", "D", weight=7)
    g.add_edge("D", "F", weight=3)
    g.add_edge("E", "F", weight=8)
    return g

In [ ]:
g = test_graph()
# Can be used for testing
#g = krackhardt_graph() 

m = # TODO
k = 0 # iteration number

# Loop until no improvement can be made through merging
while True:
    V = g.nodes
    M_com = {} # M_com is explained in sub task b)
    nodes_dict = {} # A dictionary that contains all nodes as keys and tracks their respective community

    # TODO: Initialize M_com and calculate mod_new
    
    
    # Loop until no node movement/improvement can be made
    while True:
       
        mod = mod_new

        M_com_old = copy.deepcopy(M_com)
        v_random = random.sample(V, k=len(V))
        
        # TODO: Loop over all nods and communities in order to find the best partition
    
    
    # TODO: Merge the communities with your function if an improvement is made
    
    
print("End result")
print("Modularity: " + str(mod))
plotGraph(g)

## Task 3.2: Clustering Comparison 
Now you have implemented a clustering algorithm yourself, it is time to compare performances of different clustering approaches. It is your task to compare algorithms "K-means" and "Gaussian Mixture Models" and evaluate their results.

### a) The dataset
First we create a dataset. The dataset contains samples from two overlapping Gaussian distributions as well as a uniform distribution. You can change the distributions to different values for testing and answering questions below but besides that you do not have to edit this code block.

In [ ]:
import numpy.random as npr

# Create dataset with 2 Gaussian distributions and a uniform distribution
def create_dataset(datapoints=3000):
    
    # Create dataset consisting only of zeros
    data = np.zeros([datapoints, 3])
    dist_size = datapoints//3

    # Create 2 Gaussian distributions and one uniform distribution
    gaussian_dist1 = npr.multivariate_normal(mean=[-1, 1], cov=[[2, 1],[1, 2]], size=(dist_size))
    gaussian_dist2 = npr.multivariate_normal(mean=[1, 5], cov=[[3, 1],[2, 1]], size=(dist_size))
    uniform_dist = npr.uniform([5, -3], [10, 5], size=(dist_size, 2))

    # Merge all distributions into one dataset
    data[:dist_size, :2] = gaussian_dist1
    data[:dist_size, 2] = np.asarray([0]*dist_size)
    data[dist_size:2*dist_size, :2] = gaussian_dist2
    data[dist_size:2*dist_size, 2] = np.asarray([1]*dist_size)
    data[2*dist_size:, :2] = uniform_dist
    data[2*dist_size:, 2] = np.asarray([2]*dist_size)

    return data

### b) Clustering
With a dataset in your hands, you can now define a K-means and GMM model that compute clusterings. Use the same number of max iterations for both, so they are comparable to each other. 

First you have to initialize the models and then compute clusterings on our our data.

**Notes:**
* You do not have to code anything yourself, just use the pre-defined functions.
* Think about an optimal number of clusters for each case.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

data = create_dataset(3000)

# TODO: Initialize K-Means and Gaussian Mixture models


# TODO: Fit and predict the data



### c) Plotting and evaluation
Finally you can plot the results. You can change different parameters in the code blocks above to answer the following questions:

* What kind of effect does the amount of datapoints have?
* Describe the kind of error (shape) K-means/GMM produce!
* Compare the results of K-means and GMM with different parameter settings

**TODO: Write your observations here**  

#### K-means clustering
**Note**: Depending on the amount of datapoints you have, it may take a while to display.

In [ ]:
colors = ["r", "b", "g"]

# Color the datapoints according to the kmeans prediction

for i, point in enumerate(data):
    plt.scatter(point[0],point[1],color=colors[y_kmeans[i]])
    
plt.show()

#### GMM clustering
**Note**: Depending on the amount of datapoints you have, it may take a while to display.

In [ ]:
colors = ["r", "b", "g"]

# Color the datapoints according to the gmm prediction

for i, point in enumerate(data):
    plt.scatter(point[0],point[1],color=colors[y_gmm[i]])
    
plt.show()

## References

[1] https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.quality.modularity.html
<br>[2] https://towardsdatascience.com/louvain-algorithm-93fde589f58c
<br>[3] https://miro.medium.com/max/2400/1*gglKE-X25TSwtw9kG0D5vg.png
<br>[4] https://www.statworx.com/de/blog/community-detection-with-louvain-and-infomap/